# Parte 2 - Laboratorio 4
### Sara Zavala
### Ricardo Valenzuela

## Dataset

In [372]:
import pandas as pd
import numpy as np
import seaborn as sns

from quickda.explore_data import *

In [373]:
df_init = pd.read_csv("VirusSample.csv")

In [374]:
df_init.head()

,file,api,class
0,7ff49f2f0912352416b05c010f35f402cc79feed,"IntersectRect,GetCurrentProcess,GetVersion",Virus
1,50cc6c99ec285d0db45dde07d8fdc18d9098c5b6,"GetCaretBlinkTime,CountClipboardFormats,GetCon...",Virus
2,f77c6bd4aebacd1a01d02e0cb20642ebf2d32929,"VarR8Pow,GetClipboardViewer,GetInputDesktop,Ge...",Virus
3,349c367c5b88fbb6cafae5d7109588d7250e16b5,"SetTraceCallback,CopyAcceleratorTableW,GetProc...",Virus
4,021f4aa86b520e1d606ab26699c35546bcd00c27,"SHLoadNonloadedIconOverlayIdentifiers,VarUI8Fr...",Virus


## Analisis exploratorio preliminar

In [375]:
explore(df_init, method="summarize")

,dtypes,count,null_sum,null_pct,nunique,min,25%,50%,75%,max,mean,median,std,skew
api,object,9795,0,0.0,2980,?changeGuard@QMetaObject@@SAXPAPAVQObject@@PAV...,-,-,-,"wvsprintfW,VarDecFix,GetUserDefaultLangID,GetC...",-,-,-,-
class,object,9795,0,0.0,13,Adware,-,-,-,Worms,-,-,-,-
file,object,9795,0,0.0,9795,0003103d2781125e0fd463e5042d3f26cb932f01,-,-,-,fffa9307c95121145fb056748bc830d1724f81d1,-,-,-,-


In [376]:
df_init.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9795 entries, 0 to 9794
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   file    9795 non-null   object
 1   api     9795 non-null   object
 2   class   9795 non-null   object
dtypes: object(3)
memory usage: 229.7+ KB


Solo tenemos 3 columnas iniciales, por lo que de momento no es necesario hacer un pandas profiling.

In [377]:
df_init["class"].value_counts()

Trojan        6153
Virus         2367
Backdoor       447
Worms          441
Adware         222
Agent          102
Downloader      31
Spyware         11
Ransomware      10
Riskware         4
Dropper          4
Crypt            2
Keylogger        1
Name: class, dtype: int64

Tenemos 13 diferentes clases, sin embargo, vemos tambien que el dataset esta muy desbalanceado, la diferencia entre su clase con mas incidencias con la de menos es demasiado grande. Decidimos quitar aquellas clases que tienen menos de 100 incidencias, esto ya que no queremos agregar overfitting al intentar balancear demasiado las clases que tienen pocos registros.

In [378]:
df_init.drop(df_init[df_init["class"] == "Downloader"].index, inplace=True)
df_init.drop(df_init[df_init["class"] == "Spyware"].index, inplace=True)
df_init.drop(df_init[df_init["class"] == "Ransomware"].index, inplace=True)
df_init.drop(df_init[df_init["class"] == "Riskware"].index, inplace=True)
df_init.drop(df_init[df_init["class"] == "Dropper"].index, inplace=True)
df_init.drop(df_init[df_init["class"] == "Crypt"].index, inplace=True)
df_init.drop(df_init[df_init["class"] == "Keylogger"].index, inplace=True)

In [379]:
df_init["class"].value_counts()

Trojan      6153
Virus       2367
Backdoor     447
Worms        441
Adware       222
Agent        102
Name: class, dtype: int64

Transformamos la clase a una variable cuantitativa para facilidad al crear modelos:  
    - 0: Adware  
    - 1: Agent  
    - 2: Backdoor  
    - 3: Trojan  
    - 4: Virus  
    - 5: Worms

In [380]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df_init['class'] = le.fit_transform(df_init['class'])

In [381]:
df_init["class"].value_counts()

3    6153
4    2367
2     447
5     441
0     222
1     102
Name: class, dtype: int64

Obtenemos conteo de palabras:

In [382]:
features = df_init.loc[:, df_init.columns != 'class']
target = df_init["class"]

In [383]:
features_df = pd.DataFrame(features,dtype=str)
features_df.apply(le.fit_transform)

features_df = pd.DataFrame(features_df.apply(le.fit_transform).values[:,:])
features_df

,0,1
0,4887,1240
1,3104,599
2,9414,2501
3,2027,2245
4,83,2127
...,...,...
9727,2242,1611
9728,3233,2059
9729,8390,2708
9730,7864,1209


Balancemos un poco los datos, siempre manteniendo el patron para no agregar overfitting

In [384]:
from collections import Counter
print("Tamanio completo del df tras preprocesamiento: ", len(df_init))
print("Antes del oversampling: ",Counter(target))

Tamanio completo del df tras preprocesamiento:  9732
Antes del oversampling:  Counter({3: 6153, 4: 2367, 2: 447, 5: 441, 0: 222, 1: 102})


In [385]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(sampling_strategy = {0: 2000, 1:1000, 2:3500, 4:4000, 5:3000})

x, y = sm.fit_resample(features_df, target)

oversampled_data = x

print("Luego del oversampling: ",Counter(y))

Luego del oversampling:  Counter({3: 6153, 4: 4000, 2: 3500, 5: 3000, 0: 2000, 1: 1000})


Ahora escalamos los datos con el fin de no ver tanta diferencia entre datos con las estadisticas y graficas

In [386]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(x)
features_df = pd.DataFrame(scaler.transform(x))
features_df

,0,1
0,0.502209,0.421912
1,0.318981,0.203811
2,0.967424,0.850970
3,0.208303,0.763865
4,0.008529,0.723716
...,...,...
19648,0.412188,0.240898
19649,0.163395,0.499149
19650,0.914808,0.448452
19651,0.560888,0.798571


Finalmente dividimos los datos

In [387]:
from sklearn.model_selection import train_test_split
features_train, features_test, target_train, target_test = train_test_split(features_df, y, test_size=0.3, random_state=42)

## Naive Bayes

In [388]:
from sklearn.naive_bayes import MultinomialNB
#Creacion de modelo
naiveBB = MultinomialNB()
naiveBB.fit(features_train, target_train)

MultinomialNB()

In [389]:
from sklearn import metrics
#Prueba para el conjunto de entrenamiento, esto se hace para luego comprar resultados con el conjunto de test y determinar si hay overfitting o no.
predict_train = naiveBB.predict(features_train)
print("El valor de accuracy con la data de entrenamiento es de: ", metrics.accuracy_score(predict_train, target_train))

El valor de accuracy con la data de entrenamiento es de:  0.3116958639238206


In [390]:
#Prueba para el conjunto de entrenamiento, esto se hace para luego comprar resultados con el conjunto de test y determinar si hay overfitting o no.
predict_test = naiveBB.predict(features_test)
print("El valor de accuracy con la data de test es de: ", metrics.accuracy_score(predict_test, target_test))

El valor de accuracy con la data de test es de:  0.31631614654002715


In [391]:
from sklearn.metrics import *
#Matriz de confusion
print(classification_report(target_test, predict_test))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       591
           1       0.00      0.00      0.00       299
           2       0.00      0.00      0.00      1084
           3       0.32      1.00      0.48      1865
           4       0.00      0.00      0.00      1197
           5       0.00      0.00      0.00       860

    accuracy                           0.32      5896
   macro avg       0.05      0.17      0.08      5896
weighted avg       0.10      0.32      0.15      5896



C:\Users\Zephyrus\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

C:\Users\Zephyrus\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

C:\Users\Zephyrus\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [392]:
cf = confusion_matrix(target_test, predict_test)
cf

array([[   0,    0,    0,  591,    0,    0],
       [   0,    0,    0,  299,    0,    0],
       [   0,    0,    0, 1084,    0,    0],
       [   0,    0,    0, 1865,    0,    0],
       [   0,    0,    0, 1197,    0,    0],
       [   0,    0,    0,  860,    0,    0]], dtype=int64)

In [393]:
from sklearn import tree
clfTree = tree.DecisionTreeClassifier()
clfTree.fit(features_train,target_train)

DecisionTreeClassifier()

In [394]:
#Prueba para el conjunto de entrenamiento, esto se hace para luego comprar resultados con el conjunto de test y determinar si hay overfitting o no.
predict_train = clfTree.predict(features_train)
print("El valor de accuracy con la data de entrenamiento es de: ", metrics.accuracy_score(predict_train, target_train))

El valor de accuracy con la data de entrenamiento es de:  0.9987642654648543


In [395]:
predict_test = naiveBB.predict(features_test)
print("El valor de accuracy con la data de test es de: ", metrics.accuracy_score(predict_test, target_test))

El valor de accuracy con la data de test es de:  0.31631614654002715


In [396]:
from sklearn.metrics import *
#Matriz de confusion
print(classification_report(target_test, predict_test))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       591
           1       0.00      0.00      0.00       299
           2       0.00      0.00      0.00      1084
           3       0.32      1.00      0.48      1865
           4       0.00      0.00      0.00      1197
           5       0.00      0.00      0.00       860

    accuracy                           0.32      5896
   macro avg       0.05      0.17      0.08      5896
weighted avg       0.10      0.32      0.15      5896



C:\Users\Zephyrus\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

C:\Users\Zephyrus\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

C:\Users\Zephyrus\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [397]:
cf = confusion_matrix(target_test, predict_test)
cf

array([[   0,    0,    0,  591,    0,    0],
       [   0,    0,    0,  299,    0,    0],
       [   0,    0,    0, 1084,    0,    0],
       [   0,    0,    0, 1865,    0,    0],
       [   0,    0,    0, 1197,    0,    0],
       [   0,    0,    0,  860,    0,    0]], dtype=int64)